# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.gain_function = self.__gain_gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.gain_function = self.__gain_entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.gain_function = self.__gain_missclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - ((l_c ** 2 / l_s).sum(axis=1) +
                    (r_c ** 2 / r_s).sum(axis=1)) / (l_s + r_s)[0]

    def __gain_gini(self, y):
        return 1 - ((np.bincount(y).astype('float') / y.size) ** 2).sum()

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -((l_c * (l_c / l_s).log2()).nansum(axis=1) +
                 (r_c * (r_c / r_s).log2()).nansum(axis=1))/(l_s + r_s)[0]

    def __gain_entropy(self, y):
        return -((np.bincount(y).astype(float) / y.size) *
                 (np.bincount(y).astype(float) / y.size).log2()).nansum()

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (l_c.max(axis=1) +
                    r_c.max(axis=1)) / (l_s + r_s)[0]

    def __gain_missclass(self, y):
        return 1 - (np.bincount(y).astype(float) / y.size).max()

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:max(1, int(np.sqrt(n_feature)))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:max(1, int(np.log2(n_feature)))]

    def __get_feature_ids_N(self, n_feature):
        return np.random.permutation(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Сортирует фичу x по возрастанию
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Что делает этот блок кода?
        # Обрезает с концов данные
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        # Находит индексы в массиве splitted_sorted_y, где меняется класс
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)
        if len(r_border_ids) == 0:
            return np.inf, None

        # Что делает этот блок кода?
        # Формирует матрицу, в которой фиксируются подряд идущие классы
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)
        # Что делает этот блок кода?
        # Формирует разбиения на правое и левое поддерево
        l_class_count = np.cumsum(class_increments, axis=0)

        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Считает метрику G_function для каждого разбиения и находит
        # индекс самого лучшего
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # возвращает лучшее показание G_function и значение критерия,
        # по которому будем делить
        # на правое и левое поддерево
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        if (self.max_depth is not None and depth >= self.max_depth or
                np.any(np.bincount(y) > y.shape[0] * self.sufficient_share)):
            if (y.shape[0] > 0):
                bn = np.bincount(y)
                self.tree[node_id] =\
                    (self.LEAF_TYPE, bn.argmax(), bn / y.shape[0])
        else:
            gs = np.inf
            treshold = None
            feature_id = 0

            ids = self.get_feature_ids(x.shape[1])
            for i in ids:
                gs_, treshold_ = self.__find_threshold(x[:, i], y)
                if gs_ < gs:
                    gs = gs_
                    treshold = treshold_
                    feature_id = i

            if gs == np.inf:
                bn = np.bincount(y)
                self.tree[node_id] =\
                    (self.LEAF_TYPE, bn.argmax(), bn / y.shape[0])
            else:
                x_left, x_right, y_left, y_right =\
                    self.__div_samples(x, y, feature_id, treshold)
                if (y_left.shape[0] <= self.min_samples_split or
                        y_right.shape[0] <= self.min_samples_split):
                    bn = np.bincount(y)
                    self.tree[node_id] =\
                        (self.LEAF_TYPE, bn.argmax(), bn / y.shape[0])
                else:
                    self.tree[node_id] =\
                        [self.NON_LEAF_TYPE, feature_id, treshold]
                    self.feature_importances_[ids[feature_id]] =\
                        (self.gain_function(y) * y.size - (
                         self.gain_function(y_left) * y_left.size +
                         self.gain_function(y_right) * y_right.size))
                    self.__fit_node(x_left, y_left, 2*node_id + 1, depth + 1)
                    self.__fit_node(x_right, y_right, 2*node_id + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 932 µs


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 10.9 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8968253968253969

## Подготовка данных Speed Dating Data 

In [9]:

df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', encoding='latin1')
df = df[['match','age_o', 'race_o','goal','race_o', 'round' ,'condtn',  'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha','dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o',  'prob_o','met_o']].dropna()
y = np.array(df['match']).astype('int')
X = np.array(df.drop(['match'], axis=1)).astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

my_clf = MyDecisionTreeClassifier(min_samples_split=2, sufficient_share=0.95)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [10]:

%time clf.fit(X_train, y_train)

Wall time: 17.4 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [11]:
%time my_clf.fit(X_train, y_train)

Wall time: 150 ms


## Проверка качества работы на Speed Dating Data

In [12]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.6856698451342396

In [13]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.6440885621772994

## Задание 3

In [14]:
print(np.argsort(clf.feature_importances_)[-10:])

[ 0 15 17  9  6  2 12 16 18 11]


In [15]:
print(np.argsort(my_clf.feature_importances_)[-10:])

[ 7  2  9  4  1 12 17 13 15  0]


## Задание 4

In [16]:
params = {'max_depth': range(1,30),
          'n_estimators': range(5, 100),
          'min_samples_leaf': range(1,10)}

clf = RandomForestClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=params)
random_search.fit(X, y)
print(random_search.best_params_)

{'n_estimators': 51, 'min_samples_leaf': 4, 'max_depth': 9}
